In [5]:
# Import required libraries for Reddit API data collection
import pandas as pd  # For data manipulation and analysis
import re
from datetime import datetime
import emoji
import string
from pathlib import Path
import os
from pysentimiento import create_analyzer
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")


/opt/anaconda3/envs/uoft_senti_db/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
def clean_text(text):
    """
    Remove all special characters from the text except for letters, numbers, and spaces.
    """
    return re.sub(r'[^\w\s]', '', text)


def convert_emoji_to_text(text, emoji_wrapper="emoji"):
    """
    Converts emoji in the text to descriptive text.
    
    This function uses emoji.demojize() to replace any emoji with their
    corresponding text (e.g., "😄" becomes ":smile:"). It then wraps the
    emoji description using the provided emoji_wrapper.
    
    Args:
        text (str): The original text containing emoji.
        emoji_wrapper (str): A string that will be used to wrap the emoji text.
        
    Returns:
        str: The text with emojis converted to wrapped text.
    """
    # Convert emojis to descriptive text (e.g., :smile:)
    demojized = emoji.demojize(text)
    # Define a wrapper string (e.g., " emoji ")
    wrapper = f" {emoji_wrapper} ".replace("  ", " ")
    # Replace the demojized emoji pattern :emoji_name: with the wrapped emoji_name
    # For example, ":smile:" becomes " emoji smile emoji "
    result = re.sub(r':([^:\s]+):', lambda m: wrapper + m.group(1) + wrapper, demojized)
    return result

def is_valid_word(word):
    """
    Returns True if the word is either:
    - Composed solely of punctuation
    - Contains at least one English letter (a-z or A-Z)
    """
    # Keep if word is only punctuation
    if all(ch in string.punctuation for ch in word):
        return True
    # Keep if the word contains at least one ASCII letter
    if re.search(r'[a-zA-Z]', word):
        return True
    return False

def filter_non_english(text):
    """
    Splits the text into words and filters out any word that is not an English word,
    an emoji, or punctuation.
    
    Args:
        text (str): The input text.
    
    Returns:
        str: The cleaned text.
    """
    words = text.split()
    filtered_words = [word for word in words if is_valid_word(word)]
    return ' '.join(filtered_words)

# 4. Convert created_utc to readable date format (YYYY-MM-DD)
def convert_utc_to_readable(utc_timestamp):
    """
    Convert UTC timestamp to a readable date format (YYYY-MM-DD).
    
    Args:
        utc_timestamp (float): UTC timestamp in seconds since epoch.
        
    Returns:
        str: Formatted date string or empty string if input is NaN.
    """
    
    try:
        # Handle NaN values
        if pd.isna(utc_timestamp):
            return ""
        
        # Convert to datetime object
        dt = datetime.fromtimestamp(utc_timestamp)
        
        # Format as YYYY-MM-DD HH:MM:SS
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
    except (ValueError, TypeError, OSError):
        return ""
    

In [7]:
def sentiment_analysis(master_df, text_column='body'):
    """
    Iterates through each text in the DataFrame, applies sentiment_analyzer,
    and adds two columns: sentiment_prediction and sentiment_prob.
    
    Args:
        master_df (pd.DataFrame): The DataFrame to analyze
        text_column (str): The column containing text to analyze
        
    Returns:
        pd.DataFrame: DataFrame with added sentiment columns
    """
    senti_pred = []
    senti_prob = []
    
    for index, row in master_df.iterrows():
        text = row[text_column]
        
        if pd.isna(text) or text == "":
            senti_pred.append("NEU")
            senti_prob.append({"NEU": 0.34, "NEG": 0.33, "POS": 0.33})
        else:
            result = sentiment_analyzer.predict(text)
            senti_pred.append(result.output)
            senti_prob.append(result.probas)
    
    master_df['sentiment_prediction'] = senti_pred
    master_df['sentiment_prob'] = senti_prob
    master_df['sentiment_score'] = master_df['sentiment_prob'].apply(lambda x: x.get('POS', 0) - x.get('NEG', 0))
    
    return master_df

def emotion_analysis(input_df, text_column='body'):
    emotion_pred = []
    emotion_prob = []
    
    for index, row in input_df.iterrows():
        text = row[text_column]
        result = classifier(text)
        
		# Sort results by score in descending order
        sorted_results = sorted(result[0], key=lambda x: x['score'], reverse=True)
        
        # Get top emotion and its confidence
        top_emotion = sorted_results[0]
        top_confidence = top_emotion['score']
        
        if top_confidence >= 0.7:
            # Use only top emotion if confidence >= 70%
            emotion_pred.append(top_emotion['label'])
            emotion_prob.append({top_emotion['label']: top_emotion['score']})
        else:
            # Use top 2 emotions if confidence < 70%
            top_2_emotions = sorted_results[:2]
            labels = [emotion['label'] for emotion in top_2_emotions]
            emotion_pred.append(labels)  # Store as list of 2 emotions
            emotion_prob.append({emotion['label']: emotion['score'] for emotion in top_2_emotions})

    input_df['emotion_prediction'] = emotion_pred
    input_df['emotion_prob'] = emotion_prob

    return input_df

def preprocess_dataframe(df):
    """
    Apply all preprocessing steps to a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to preprocess
        
    Returns:
        pd.DataFrame: The preprocessed DataFrame
    """
    # Make a copy to avoid modifying the original
    processed_df = df.copy()
    
    # 1. Keep only posts with URLs starting with UofT comments
    processed_df = processed_df[processed_df['url'].str.startswith('https://www.reddit.com/r/UofT/comments/')].reset_index(drop=True).copy()
    
    # 2. Clean body text: lowercase, strip, remove special chars except spaces
    processed_df['body'] = processed_df['body'].str.lower()
    processed_df['body'] = processed_df['body'].str.strip()
    
    # 3. Remove special characters except spaces
    processed_df['body'] = processed_df['body'].apply(clean_text)
    
    # 4. Converts emoji in the text to descriptive text
    processed_df['body'] = processed_df['body'].apply(convert_emoji_to_text)
    
    # 5. Filter out non-English words
    processed_df['body'] = processed_df['body'].apply(filter_non_english)
    
    # 6. Convert created_utc to readable date format
    processed_df['created_utc'] = processed_df['created_utc'].apply(convert_utc_to_readable)
    
    # 7. Preprocess the body column using pysentimiento (optional - currently commented out)
    # processed_df['body'] = processed_df['body'].apply(preprocess_tweet)
    
    return processed_df


def process_all_monthly_submissions(folder_path, senti=True, emo=True):
    """
    Process all CSV files in the monthly submissions folder.
    Apply preprocessing and sentiment analysis to each file.
    
    Args:
        folder_path (str): Path to the folder containing CSV files
        
    Returns:
        pd.DataFrame: Combined DataFrame with all processed submissions
    """
    # Get list of all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    csv_files.sort()  # Sort to process in order
    
    print(f"Found {len(csv_files)} CSV files to process")
    
    all_processed_data = []
    
    for i, filename in enumerate(csv_files):
        if filename.startswith('top_100_reddits_'):
            continue  # Skip the combined file if it exists
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file {i+1}/{len(csv_files)}: {filename}")
        
        try:
            # Load the CSV file
            df = pd.read_csv(file_path)
            print(f"  - Loaded {len(df)} rows")
            
            # Apply preprocessing
            processed_df = preprocess_dataframe(df)
            print(f"  - After filtering: {len(processed_df)} rows")
            
            # Apply sentiment analysis
            if len(processed_df) > 0:
                if senti:
                    processed_df = sentiment_analysis(processed_df, text_column='body')
                    print(f"  - Sentiment analysis completed")

                if emo:
                    processed_df = emotion_analysis(processed_df, text_column='body')
                    print(f"  - Emotion analysis completed")
                
                # Add source file information
                processed_df['source_file'] = filename
                
                all_processed_data.append(processed_df)
            else:
                print(f"  - No data remaining after filtering")
                
        except Exception as e:
            print(f"  - Error processing {filename}: {str(e)}")
            continue
    
    # Combine all processed data
    if all_processed_data:
        combined_df = pd.concat(all_processed_data, ignore_index=True)
        print(f"\nProcessing complete! Combined dataset has {len(combined_df)} rows")
        return all_processed_data, combined_df
    else:
        print("\nNo data was successfully processed")
        return pd.DataFrame()


# Example usage:
# processed_monthly_data = process_all_monthly_submissions('individual_submissions_month')
# processed_monthly_data.to_excel('processed_monthly_submissions_with_sentiment.xlsx', index=False)

In [8]:
# Process all monthly submissions
print("Starting to process all monthly submission CSV files...")
today = datetime.now().strftime('%Y%m')
folder_path = f'monthly_top100/{today}'
all_processed_data, processed_monthly_data = process_all_monthly_submissions(folder_path, senti=True, emo=True)

# Display summary statistics
if len(processed_monthly_data) > 0:
    print(f"\n=== PROCESSING SUMMARY ===")
    print(f"Total processed rows: {len(processed_monthly_data)}")
    print(f"Unique submissions: {processed_monthly_data['submission_title'].nunique()}")
    print(f"Date range: {processed_monthly_data['created_utc'].min()} to {processed_monthly_data['created_utc'].max()}")
    
    # Sentiment distribution
    print(f"\n=== SENTIMENT DISTRIBUTION ===")
    sentiment_counts = processed_monthly_data['sentiment_prediction'].value_counts()
    print(sentiment_counts)
    print(f"Sentiment percentages:")
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(processed_monthly_data)) * 100
        print(f"  {sentiment}: {percentage:.1f}%")
else:
    print("No data was processed successfully.")

Starting to process all monthly submission CSV files...
Found 98 CSV files to process
Processing file 1/98: submission_0.csv
  - Loaded 90 rows
  - After filtering: 89 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 2/98: submission_1.csv
  - Loaded 27 rows
  - After filtering: 26 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 3/98: submission_10.csv
  - Loaded 61 rows
  - After filtering: 60 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 4/98: submission_11.csv
  - Loaded 33 rows
  - After filtering: 33 rows
  - Sentiment analysis completed
  - Error processing submission_11.csv: The expanded size of the tensor (553) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 553].  Tensor sizes: [1, 514]
Processing file 5/98: submission_12.csv
  - Loaded 42 rows
  - After filtering: 42 rows
  - Sentiment analysis completed
  - Emotion analysis compl

In [9]:
today = datetime.now().strftime('%Y%m')
file_name = Path(f'top_100_reddits_{today}.csv')
processed_monthly_data.to_csv(folder_path / file_name, index=False)

In [10]:
sentiment_counts = processed_monthly_data['sentiment_prediction'].value_counts()


In [12]:
processed_monthly_data

,submission_title,author,id,url,created_utc,body,type,score,sentiment_prediction,sentiment_prob,sentiment_score,emotion_prediction,emotion_prob,source_file
0,these types of questions are actually so dumb ...,buckbuck5645,nes2v1g,https://www.reddit.com/r/UofT/comments/1njph3e...,2025-09-17 17:35:18,this might be one of the dumbest questions ive...,comment,274,NEG,"{'NEG': 0.978022575378418, 'NEU': 0.0189272090...",-0.974972,"[disgust, sadness]","{'disgust': 0.36396047472953796, 'sadness': 0....",submission_0.csv
1,these types of questions are actually so dumb ...,SNG_Blitzy,nesxtjv,https://www.reddit.com/r/UofT/comments/1njph3e...,2025-09-17 20:29:13,its essentially the same thing but its crucial...,comment,128,NEU,"{'NEG': 0.028543880209326744, 'NEU': 0.9490432...",-0.006131,neutral,{'neutral': 0.8884442448616028},submission_0.csv
2,these types of questions are actually so dumb ...,BromineFromine,nes8way,https://www.reddit.com/r/UofT/comments/1njph3e...,2025-09-17 18:07:21,when you put a vengeful pedant in charge of yo...,comment,61,NEG,"{'NEG': 0.9388996958732605, 'NEU': 0.057808689...",-0.935608,anger,{'anger': 0.9504784345626831},submission_0.csv
3,these types of questions are actually so dumb ...,onlyonequickquestion,neskxvd,https://www.reddit.com/r/UofT/comments/1njph3e...,2025-09-17 19:15:21,well maybe in a technical or legal sense there...,comment,40,NEG,"{'NEG': 0.801746666431427, 'NEU': 0.1943267583...",-0.797820,"[neutral, anger]","{'neutral': 0.5479668378829956, 'anger': 0.251...",submission_0.csv
4,these types of questions are actually so dumb ...,BackgroundBench530,netfy6p,https://www.reddit.com/r/UofT/comments/1njph3e...,2025-09-17 22:14:05,im a criminologist even the correct answer is ...,comment,20,NEU,"{'NEG': 0.33407503366470337, 'NEU': 0.63717758...",-0.305328,"[anger, fear]","{'anger': 0.4802381694316864, 'fear': 0.261093...",submission_0.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,So what’s with the people protesting abortions...,OdditiesInOntario,nfavgkm,https://www.reddit.com/r/UofT/comments/1nlnfuw...,2025-09-20 15:56:15,which is wholly and totally irrelevant here be...,comment,1,NEU,"{'NEG': 0.3910699188709259, 'NEU': 0.600081980...",-0.382222,anger,{'anger': 0.9745969176292419},submission_96.csv
2538,So what’s with the people protesting abortions...,torontopeter,nf8hx2h,https://www.reddit.com/r/UofT/comments/1nlnfuw...,2025-09-20 08:26:53,not disagreeing with that these despicable peo...,comment,-3,NEG,"{'NEG': 0.9640244245529175, 'NEU': 0.033359553...",-0.961408,disgust,{'disgust': 0.9778687357902527},submission_96.csv
2539,So what’s with the people protesting abortions...,ThatGenericName2,nf8j5uv,https://www.reddit.com/r/UofT/comments/1nlnfuw...,2025-09-20 08:35:19,ok what were you saying with your original com...,comment,5,NEU,"{'NEG': 0.01563001424074173, 'NEU': 0.96736568...",0.001374,surprise,{'surprise': 0.8578999042510986},submission_96.csv
2540,So what’s with the people protesting abortions...,torontopeter,nf8jk8d,https://www.reddit.com/r/UofT/comments/1nlnfuw...,2025-09-20 08:37:55,correcting your mistake re the charter,comment,-1,NEU,"{'NEG': 0.32758137583732605, 'NEU': 0.66043359...",-0.315596,"[sadness, anger]","{'sadness': 0.4839746654033661, 'anger': 0.256...",submission_96.csv
